In [ ]:
import os,threading
from pathlib import Path

from flask import Flask, request, abort
import logging
from logging.handlers import RotatingFileHandler

import IMdb_crawler
import kimo
import rottenTomatos

from linebot import (
    LineBotApi, WebhookHandler
)
from linebot.exceptions import (
    InvalidSignatureError
)
from linebot.models import (
    MessageEvent, TextMessage, ImageMessage, TextSendMessage, ImageSendMessage,ImageCarouselColumn,MessageTemplateAction,
    TemplateSendMessage,ImageCarouselTemplate,URITemplateAction,ImagemapArea,QuickReplyButton,QuickReply,PostbackAction,
    MessageAction
)
import json

from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

import pymysql

app = Flask(__name__)

#app.config['DEBUG'] = True

#與mysql進行連線
conn = pymysql.Connect(host='chatbot_db',
                        port=3306,
                        user='root',
                        passwd='123456',
                        db='TFB103d_azure',
                        charset='utf8')
cur = conn.cursor()

secretFile = json.load(open("secretFile.txt",'r'))
channelAccessToken = secretFile['channelAccessToken']
channelSecret = secretFile['channelSecret']

line_bot_api = LineBotApi(channelAccessToken)
handler = WebhookHandler(channelSecret)
ngrokUrl ='change your ngrok link'



@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        print("Invalid signature. Please check your channel access token/channel secret.")
        abort(400)

    return 'OK'


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    
    #判斷session是否有同一個user_id
    '''
    if(event.source.user_id in userSessionDict):
        sessionActor=userSessionDict[event.source.user_id]
        sessionActor += event.message.text
        userSessionDict[event.source.user_id]=sessionActor
        returnMessage = TextSendMessage(sessionActor)
    else:
        userSessionDict[event.source.user_id]=event.message.text
        returnMessage=TextSendMessage(event.message.text)
    '''
    message = TextSendMessage(
        text="未查詢到電影，請再提供圖片進行查詢。",
        quick_reply=QuickReply(
            items=[
                QuickReplyButton(
                    action=MessageAction(label="重新查詢",data="@重新查詢")
                ),
            ]))
    if event.message.text == '@熱門電影':
        message = hotMovie(event)
    elif event.message.text == '@最新電影':
        message = newMovie(event)
    elif event.message.text == '@重新查詢':
        userSessionDict[event.source.user_id]=set()
        message = TextSendMessage("已清除資訊，請重新查詢。")
    elif event.message.text == '@訂票連結':
        message = movieLink(event)
    elif '#' in event.message.text:
        message = movieDesc(event.message.text)
    
    line_bot_api.reply_message(
        event.reply_token,message
        )

@handler.add(MessageEvent, message=ImageMessage)
def handle_image_message(event):
    message_id = event.message.id
    app.logger.info("message_id: " + message_id)
    # 從message_id取得image的內容
    message_content = line_bot_api.get_message_content(message_id)

    with open(Path(f"static/images/{message_id}.jpg").absolute(), "wb") as f:
        # 存入本機位置
        for chunk in message_content.iter_content():
            f.write(chunk)
    
    
    actorList=faceDetection(message_id)
    
    if(actorList == None):
        line_bot_api.reply_message(event.reply_token, TextSendMessage("電影資料抓取中，請稍候。"))
    elif(len(actorList) == 0):
        line_bot_api.reply_message(event.reply_token, TextSendMessage("未偵測到演員，請重新上傳。"))
    else:
        resultList=[]
        #判斷是不是新的使用者，若是同一使用者，則將上次查詢的結果進行結合
        if(event.source.user_id in userSessionDict):
            sessionActorSet=userSessionDict[event.source.user_id]
            resultSet = sessionActorSet | set(actorList)
            userSessionDict[event.source.user_id]=resultSet
            resultList=list(resultSet)
        else:
            userSessionDict[event.source.user_id]=set(actorList)
            resultList = actorList

        if(len(resultList) == 1):
            selectSql='''select movie_name from movie_actor_ref where actor_name in ('%s') group by movie_name having count(actor_name) = %s LIMIT 10;''' % (resultList[0],len(resultList))
        else:
            selectSql='''select movie_name from movie_actor_ref where actor_name in %s group by movie_name having count(actor_name) = %s LIMIT 10;''' % (tuple(resultList),len(resultList))
        print(selectSql)
        cur.execute(selectSql)
        result_set = cur.fetchall()
        movieList=[]
        for row in result_set:
            movieList.append(str(row[0]))
        
        print(len(movieList))
        if(len(movieList) == 0):
            message = TextSendMessage(
                        text="未查詢到電影，請再提供圖片進行查詢，或點選重新查詢清空重查。",
                        quick_reply=QuickReply(
                            items=[
                                QuickReplyButton(
                                    action=MessageAction(label="重新查詢",text="@重新查詢")
                                ),
                            ]))
            line_bot_api.reply_message(event.reply_token, message)
        elif(len(movieList) == 1):
            updateClickSQL="UPDATE movie SET click_count = click_count + 1 where movie_name = %s ;"
            cur.execute(updateClickSQL,(movieList[0]))
            conn.commit()
            
            selectMovieSql='''select movie_name,imdb_rating,tomato_rating,kimo_rating,movie_desc from movie where movie_name = '%s' ;''' % movieList[0]
            cur.execute(selectMovieSql)
            result_set = cur.fetchall()
            score =float()
            for row in result_set:
                score = float(row[1]+row[2]+row[3]) / 3
            messageList = movieDesc(movieList[0])
            messageList.append(TextSendMessage("您查詢的電影名稱為：\n%s \n平均分數：%.1f" % (movieList[0],score)))
            
            line_bot_api.reply_message(event.reply_token, messageList)
        else:
            columnsList=[]
            for movie in movieList:
                updateClickSQL="UPDATE movie SET click_count = click_count + 1 where movie_name = %s ;"
                cur.execute(updateClickSQL,(movie))
                conn.commit()
                
                selectMovieSql='''select movie_id,link,movie_name from movie where movie_name = %s;'''
                cur.execute(selectMovieSql,(movie))
                movie_results = cur.fetchall()
                for row in movie_results:
                    uuid=row[0]
                    link=row[1]
                    labelStr=row[2]
                    textStr = "#"+labelStr
                    if(len(labelStr)>12):
                        labelStr=labelStr[:9]+"..."
                    image_cc=ImageCarouselColumn(
                        image_url=ngrokUrl+f"static/images/{uuid}.jpg",
                        action=MessageTemplateAction(
                                label=labelStr,
                                text=textStr
                            )
                    )
                    columnsList.append(image_cc)

            message = [TemplateSendMessage(
                alt_text='電影列表',
                template=ImageCarouselTemplate(
                    columns=columnsList
                )),
                TextSendMessage(
                    text = "查詢結果如上列表：\n可再上傳圖片查詢，或是點擊圖片查詢電影資訊。"
                )]

            line_bot_api.reply_message(event.reply_token, message)
    


    


def faceDetection(messageId):
    # Set API key.
    subscription_key = 'change your Azure key'
    # Set endpoint.
    endpoint = 'change your Azure endpoint'
    # Call API
    computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))
    print("===== Detect Domain-specific Content - remote =====")
    # URL of one or more celebrities
    #remote_image_url_celebs = "http://t0.gstatic.com/licensed-image?q=tbn:ANd9GcQIkdWP0IRaCfRaT029JqcMiHNpXbkUQURn_eZAziIL7K48B2kfIlRehGIof2a3"
    # Call API with content type (celebrities) and URL
    detect_domain_results_celebs_remote = computervision_client.analyze_image_by_domain_in_stream(
        "celebrities", open(f"./static/images/{messageId}.jpg", "rb"))
    
    nameList = list()
    # Print detection results with name
    print("Celebrities in the remote image:")
    if len(detect_domain_results_celebs_remote.result["celebrities"]) == 0:
        print("No celebrities detected.")
    else:
        for celeb in detect_domain_results_celebs_remote.result["celebrities"]:
            app.logger.info(celeb["name"])
            nameList.append(celeb["name"])
            #確認資料庫中已經爬過，已爬過便不再爬
            selectSql='''select count(actor_name) from movie_actor_ref where actor_name = %s ;'''
            cur = conn.cursor()
            cur.execute(selectSql,celeb["name"])
            number_of_actor = cur.fetchone()[0]
            if(int(number_of_actor) == 0):
                def movie_thread():
                    try:
                        IMdb_crawler.IMDb_crawler_by_python(celeb["name"],cur)
                        kimo.KiMoScore_Desc(celeb["name"],cur)
                        rottenTomatos.rotten_tomato_score(celeb["name"],cur)
                    finally:
                        conn.commit()
                # 建立一個子執行緒
                t = threading.Thread(target = movie_thread)

                # 執行該子執行緒
                t.start()
                
                return None
                
   
    return nameList

# 熱門電影排行
def hotMovie(event):
    try:
        sql = '''SELECT movie_id,link,movie_name
                 FROM movie
                 ORDER BY click_count DESC
                 LIMIT 5;
               '''
        cur.execute(sql)
        result = cur.fetchall()
        ans = ''
        columnsList=[]
        for row in result:
            movie_id=row[0]
            link=row[1]
            labelStr=row[2]
            textStr = "#"+labelStr
            if(len(labelStr)>12):
                labelStr=labelStr[:9]+"..."
            image_cc=ImageCarouselColumn(
                        image_url=ngrokUrl+f"static/images/{movie_id}.jpg",
                        action=MessageTemplateAction(
                                label=labelStr,
                                text=textStr
                            )
                    )
            columnsList.append(image_cc)
            #ans += str(name[0]+1)+ " " + str(name[1]).split(',')[0].split('\'')[1] + '\n'
        #ans = ans[:-1]
        return [TemplateSendMessage(
                alt_text='熱門電影排行',
                template=ImageCarouselTemplate(
                    columns=columnsList
                )),
                TextSendMessage(
                    text = "查詢結果如上：\n可再上傳圖片查詢，或是點擊圖片查詢電影資訊。"
                )]
    except:
        return TextSendMessage(text='發生錯誤！')

# 最新上映電影排行
def newMovie(event):
    try:
        sql = '''SELECT movie_id,link,movie_name
                 FROM movie
                 ORDER BY release_date DESC
                 LIMIT 5;
               '''
        cur.execute(sql)
        result = cur.fetchall()
        ans = ''
        columnsList=[]
        for row in result:
            movie_id=row[0]
            link=row[1]
            labelStr=row[2]
            textStr = "#"+labelStr
            if(len(labelStr)>12):
                labelStr=labelStr[:9]+"..."
            image_cc=ImageCarouselColumn(
                        image_url=ngrokUrl+f"static/images/{movie_id}.jpg",
                        action=MessageTemplateAction(
                                label=labelStr,
                                text=textStr
                            )
                    )
            columnsList.append(image_cc)
            #ans += str(name[0]+1)+ " " + str(name[1]).split(',')[0].split('\'')[1] + '\n'
        #ans = ans[:-1]
        return [TemplateSendMessage(
                alt_text='熱門電影排行',
                template=ImageCarouselTemplate(
                    columns=columnsList
                )),
                TextSendMessage(
                    text = "查詢結果如上：\n可再上傳圖片查詢，或是點擊圖片查詢電影資訊。"
                )]
    except:
        return TextSendMessage(text='發生錯誤！')

def movieLink(event):
    try:
        columnsList=[]
        image_aa=ImageCarouselColumn(
                    image_url=ngrokUrl+f"static/images/weishow.JPG",
                    baseSize={"width":100, "height":100},
                    action=URITemplateAction(
                            label="華納威秀",
                            uri="https://www.vscinemas.com.tw/"
                        )
                )
        image_bb=ImageCarouselColumn(
                    image_url=ngrokUrl+f"static/images/ez.JPG",
                    action=URITemplateAction(
                            label="ez訂",
                            uri="https://www.ezding.com.tw/"
                        )
                )
        image_cc=ImageCarouselColumn(
                    image_url=ngrokUrl+f"static/images/amba.JPG",
                    action=URITemplateAction(
                            label="國賓影城",
                            uri="https://www.ambassador.com.tw/"
                        )
                )
        columnsList.append(image_aa)
        columnsList.append(image_bb)
        columnsList.append(image_cc)
        #ans += str(name[0]+1)+ " " + str(name[1]).split(',')[0].split('\'')[1] + '\n'
        #ans = ans[:-1]
        return [TemplateSendMessage(
                alt_text='訂票連結',
                template=ImageCarouselTemplate(
                    columns=columnsList
                )),
                TextSendMessage(
                    text = "點擊圖片即可進入訂票網站。"
                )]
    except:
        return TextSendMessage(text='發生錯誤！')
        
        
def movieDesc(movie_name):
    try:
        
        sql = '''SELECT movie_id,link,movie_name,movie_desc
                 FROM movie
                 where movie_name = %s
                 ORDER BY release_date DESC
                 LIMIT 5;
               '''
        movie_name = movie_name.replace('#', '')
        cur.execute(sql,(movie_name))
        result = cur.fetchall()
        ans = ''
        columnsList=[]
        for row in result:
            movie_id=row[0]
            link=row[1]
            labelStr=row[2]
            textStr = "#"+labelStr
            if(len(labelStr)>12):
                labelStr=labelStr[:9]+"..."
            image_cc=ImageCarouselColumn(
                        image_url=ngrokUrl+f"static/images/{movie_id}.jpg",
                        action=URITemplateAction(
                                label=labelStr,
                                uri=link
                            )
                    )
            columnsList.append(image_cc)
            #ans += str(name[0]+1)+ " " + str(name[1]).split(',')[0].split('\'')[1] + '\n'
        #ans = ans[:-1]
        return [TemplateSendMessage(
                alt_text='電影資訊',
                template=ImageCarouselTemplate(
                    columns=columnsList
                )),
                TextSendMessage(
                    text = "電影簡介為：%s" % row[3]
                ),]
    except:
        return TextSendMessage(text='發生錯誤！')    
        
if __name__ == "__main__":
    userSessionDict = dict()
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s:%(lineno)d][%(levelname)s][%(thread)d] - %(message)s")
    loghandler = RotatingFileHandler("flask.log", maxBytes=1024000, backupCount=10)
    app.logger.addHandler(loghandler)
    app.logger.setLevel(logging.INFO)
    loghandler.setFormatter(formatter)
    #app.run(host='0.0.0.0',port='5001',debug=True,use_reloader=False)
    app.run(host='0.0.0.0',port='5000')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://172.20.0.3:5001/ (Press CTRL+C to quit)
[2021-09-11 14:45:46,072] INFO in 906307842: Request body: {"destination":"U6eec9b973b652e4e5f59f6b271643d55","events":[{"type":"message","message":{"type":"image","id":"14729837423841","contentProvider":{"type":"line"}},"timestamp":1631371545585,"source":{"type":"user","userId":"U8f34e9530d61b12ca3d938b6f587e183"},"replyToken":"776c9907bb0f4398987efe60f77781de","mode":"active"}]}
[2021-09-11 14:45:46,076] INFO in 906307842: message_id: 14729837423841


===== Detect Domain-specific Content - remote =====


[2021-09-11 14:45:47,635] INFO in 906307842: Tom Hardy
172.20.0.4 - - [11/Sep/2021 14:45:47] "POST / HTTP/1.1" 200 -


Celebrities in the remote image:
select movie_name from movie_actor_ref where actor_name in ('Tom Hardy') group by movie_name having count(actor_name) = 1 LIMIT 10;
2


...

Celebrities in the remote image:
select movie_name from movie_actor_ref where actor_name in ('Tom Hardy') group by movie_name having count(actor_name) = 1 LIMIT 10;
2


172.20.0.4 - - [11/Sep/2021 14:52:39] "GET /static/images/84b965d3-12f6-11ec-8fea-257b97825445.jpg HTTP/1.1" 404 -
172.20.0.4 - - [11/Sep/2021 14:52:39] "GET /static/images/1e60e8ca-12fa-11ec-8579-257b97825445.jpg HTTP/1.1" 404 -
172.20.0.4 - - [11/Sep/2021 14:52:39] "GET /static/images/6c16dd5d-12fd-11ec-8b91-15eaf2e80c3d.jpg HTTP/1.1" 404 -
172.20.0.4 - - [11/Sep/2021 14:52:39] "GET /static/images/87a78c6a-12f6-11ec-97c7-257b97825445.jpg HTTP/1.1" 404 -
172.20.0.4 - - [11/Sep/2021 14:52:39] "GET /static/images/ac2812d1-1179-11ec-abe8-956c67822b7c.jpg HTTP/1.1" 404 -
172.20.0.4 - - [11/Sep/2021 14:52:39] "GET /static/images/8a15ef89-12f6-11ec-8cc9-257b97825445.jpg HTTP/1.1" 404 -
172.20.0.4 - - [11/Sep/2021 14:52:39] "GET /static/images/c1852205-12fa-11ec-b6d4-257b97825445.jpg HTTP/1.1" 404 -
172.20.0.4 - - [11/Sep/2021 14:52:39] "GET /static/images/9414981c-12f6-11ec-9daf-257b97825445.jpg HTTP/1.1" 404 -
[2021-09-11 14:52:41,622] INFO in 906307842: Request body: {"destination":"U6eec

===== Detect Domain-specific Content - remote =====


172.20.0.4 - - [11/Sep/2021 14:54:35] "POST / HTTP/1.1" 200 -


Celebrities in the remote image:
No celebrities detected.


In [2]:
!pip install --upgrade azure-cognitiveservices-vision-computervision
!pip install bs4
!pip install pymysql

     |████████████████████████████████| 85 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 45 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 1.1 MB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1273 sha256=a1573b173c95987b1a31e320179ba39d7a1c4828a7d7af12b6cc0a12c1d43777
  Stored in directory: /home/jovyan/.cache/pip/wheels/73/2b/cb/099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4
